In [ ]:
import random
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#
# HELPERS
#

def get_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
def same_sign(a,b):
  boo = True
  if a == 0 and b == 0:
    return boo
  if (a > 0 and b > 0) or (a < 0 and b < 0):
    boo = True
  else:
    boo = False
  return boo

def is_flat_list(input_list):
    return isinstance(input_list, list) and all(isinstance(item, (int, float, str)) for item in input_list)

def vector_req():
  secInp = input('Row Entries')
  input_list = secInp.split(',')
  integer_list = [int(num) for num in input_list]
  return integer_list

def matrix_req():
  inp = str(input('What are the dimensions of your matrix?'))
  xIDX = inp.find('x')
  rows = int(inp[:xIDX])
  cols = int(inp[xIDX+1:])
  return np.array([vector_req() for _ in range(rows)])


In [ ]:
#
# VECTOR OPERATIONS
#


def magnitude(u):
	sum = 0
	for num in u:
		sum += num**2
	return np.sqrt(sum)

def vec_subtract(a, b):
    return [x - y for x, y in zip(a, b)]

def vec_scale(a, scalar):
    return [x * scalar for x in a]

def vec_add(a, b):
    return [x + y for x, y in zip(a, b)]

def dotProd(u,v):
	# where u and v are 1xm and mx1 respectively in the form u, v = [, , ,]
	sum = 0
	for i in range(len(v)):
		sum += u[i]*v[i]
	return sum

def is_orthogonal(u,v):
  if dotProd(u,v) == 0:
    print(np.array(u),'&',np.array(v), 'are orthogonal.')
  else:
    print(np.array(u),'&',np.array(v), 'are not orthogonal.', np.array(u),'o',np.array(v), '=', np.array(dotProd(u,v)))

def dist(u,v):
  return magnitude(vec_subtract(u,v))

def proj(h, v): # the projection of h onto v
    if is_flat_list(v):
      dot = dotProd(h, v)
      magsq = magnitude(v)**2
      coeff = dot/magsq
      final = []
      final = vec_scale(v, coeff)
      return np.array(final)
    else:
      final_mat = []
      v_T = transpose(v)
      for col in v_T:
        dot = dotProd(h, col)
        magsq = magnitude(col)**2
        coeff = dot/magsq
        new_col = vec_scale(col, coeff)
        final_mat.append(new_col)
        f_T = transpose(final_mat)
      return add_columns(f_T)

"""
def orthogonal_basis(A):
  A_T = transpose(A)
  B = []
  B.append(A_T[0])
  v1 = proj(A_T[1], B[0]).tolist()
  B.append(vec_subtract(A_T[1],v1))
  for i in range(2,len(A_T)):
    projLT = []
    for j in range(i):
      projec = proj(A_T[i], B[j])
      projLT.append(projec)
    proj_T = vec_scale(add_columns(transpose(projLT)), -1)
    newBcol = vec_add(A_T[i], proj_T)
    B.append(newBcol)
  final = transpose(B)
  return np.array(final)
"""

'     \ndef orthogonal_basis(A):\n  A_T = transpose(A)\n  B = []\n  B.append(A_T[0])\n  v1 = proj(A_T[1], B[0]).tolist()\n  B.append(vec_subtract(A_T[1],v1))\n  for i in range(2,len(A_T)):\n    projLT = []\n    for j in range(i):\n      projec = proj(A_T[i], B[j])\n      projLT.append(projec)\n    proj_T = vec_scale(add_columns(transpose(projLT)), -1)\n    newBcol = vec_add(A_T[i], proj_T)\n    B.append(newBcol)\n  final = transpose(B)\n  return np.array(final)\n'

In [ ]:
#
# MODIFYING MATRICES
#

def matrix_print(A, name):
	print()
	print(name + ' = ')
	print()
	for row in A:
		print(row)
	print()

def matrix_scale(A, lam):
    B = []
    for row in A:
        newRow = vec_scale(row, lam)
        B.append(newRow)
    return B

def matrix_normalize(A):
	A_T = transpose(A)
	for i in range(len(A_T)):
		A_T[i] = vec_scale(A_T[i], (1/magnitude(A_T[i])))
	return transpose(A_T)

def matrix_denormalize(A):
	A_T = transpose(A)
	for i in range(len(A_T)):
		A_T[i] = vec_scale(A_T[i], (magnitude(A_T[i])))
	return transpose(A_T)

def switchRows(A, i , j):
  holder = [0]*len(A[0])
  holdinglt = []
  holdinglt.append(A[i])
  A[i] = holder
  holdinglt.append(A[j])
  A[j] = holder
  A[i] = holdinglt[1]
  A[j] = holdinglt[0]
  return A

def add_columns(A):
  new_mat =[]
  for row in A:
    sum = 0
    for  i in range(len(row)):
      sum += row[i]
    new_mat.append(sum)
  fin = new_mat
  return fin

def subtract_columns(A):
  new_mat =[]
  for row in A:
    sum = 0
    for  i in range(len(row)):
      if i == 0:
        sum += row[i]
      else:
        sum = sum - row[i]
    new_mat.append(sum)
  fin = new_mat
  return fin

def drop_zeros(A):
  for i in range(len(A)):
    if A[i] == [0]*len(A):
      del A[i]
  return A


def pivot_position(a1):
  idx = 0
  if isinstance(a1, np.ndarray):
    a1 = a1.tolist()
  if a1 == [0]*len(a1):
    return -1
  for i in range(len(a1)):
    if a1[i] != 0:
      idx = i
      break
  return idx

def pivot_positions(A):
  final = []
  for row in A:
    final.append(pivot_position(row))
  return final

def pivot_arrange(A):
  C = A
  for i in range(len(A)):
    pivpos = pivot_position(A[i])
    if pivpos > 0:
      C = switchRows(A, i, pivpos)
  pivotdict = {}
  for i in range(len(C)):
    pivotdict[i] = pivot_position(C[i])
    sorted_dict = dict(sorted(pivotdict.items(), key=lambda item: (abs(item[1]), item[1])))
  fin = []
  for key in sorted_dict:
    fin.append(C[key])
  check = []
  if [0]*len(A) in fin:
    count = fin.count([0]*len(A))
    for row in fin:
      if row != [0]*len(A):
        check.append(row)
    for i in range(count):
      check.append([0]*len(A))
    return check
  else:
    return fin

def isUpper(matrix):
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    for i in range(num_rows):
        for j in range(num_cols):
            if i > j and abs(matrix[i][j]) != 0:
                return False

    return True

def identity(n):
  zero_mat = [[0] *n for _ in range(n)]
  for i in range(n):
      zero_mat[i][i] = 1
  return zero_mat


In [ ]:
#
# DECOMPOSITIONS
#

def is_linearly_independent(A):
  if len(A[0]) > len(A):
    print('The columns of', np.array(A), 'are not linearly independent, because n > m.' )
  A1, refA = rref(A)
  idealPivs = np.arange(0, len(A), 1)
  refPivs = np.array(pivot_positions(refA.tolist()))
  if np.array_equal(refPivs, idealPivs):
    if determinant(refA) != 0:
      print('The columns of')
      print(np.array(A))
      print(' are linearly indepdent because there is a nonzero pivot in each row.' )
    else:
      print('The columns of')
      print(np.array(A))
      print(' are not linearly indepdent because there is not a nonzero pivot in each row.' )


def rank(A):
  if isinstance(A,np.ndarray):
    A = A.tolist()
  un, basis = rref(A)
  pracpivs = pivot_positions(basis)
  if -1 in pracpivs:
    negcount = pracpivs.count(-1)
    return len(pracpivs) - negcount
  else:
    return len(pracpivs)

def nullity(A):
  r = rank(A)
  return len(A[0]) - r

def LU(D):
  U, non = rref(D)
  kdict = get_k()
  L = identity(len(D))
  for j in kdict.keys():
    k1, k2 = j
    if kdict[j] == 0:
      L[k1][k2] = 0
    else:
      L[k1][k2] = -1*kdict[j]
  return np.array(D), np.array(L), U

def transpose(A):
    if isinstance(A, list):
        A_T = [[row[i] for row in A] for i in range(len(A[0]))]
    else:
        A_T = np.transpose(A)
    return A_T

def orthogonal_basis(A):
    A_T = transpose(A)
    B = []
    B.append(A_T[0])
    for i in range(1, len(A_T)):
        projec = proj(A_T[i], B)
        newBcol = vec_subtract(A_T[i], projec)
        B.append(newBcol)
    final = transpose(B)
    return np.array(final)

def orthonormal_basis(A):
  return matrix_normalize(orthogonal_basis(A))

def QR(A):
  Q = orthonormal_basis(A)
  Q_T = transpose(Q)
  R = matmult(Q_T,A)
  return Q, R

In [ ]:
np.array([[1,0,0],[1,1,0],[1,1,1],[1,1,1]])

array([[1, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1]])

In [ ]:
np.array(transpose([[1,0,0],[1,1,0],[1,1,1],[1,1,1]]))

array([[1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 1]])

In [ ]:
orthogonal_basis([[1,0,0],[1,1,0],[1,1,1],[1,1,1]])

IndexError: ignored

In [ ]:
#
# REDUCED ROW ECHELON FORM
#

k = {}
def rref(mat):
    """
    Changes mat into Reduced Row-Echelon Form.
    """
    # Let's do the forward step first.
    # At the end of this for loop, the matrix is in Row-Echelon format.
    global k
    for i in range(min(len(mat), len(mat[0]))):
        # Every iteration, ignore one more row and column.
        for r in range(i, len(mat)):
            # Find the first row with a nonzero entry in the first column.
            zero_row = mat[r][i] == 0
            if zero_row:
                continue
            # Swap the current row with the first row.
            mat[i], mat[r] = mat[r], mat[i]
            # Add multiples of the new first row to lower rows so that the lower
            # entries of the first column are zero.
            first_row_first_col = mat[i][i]
            for rr in range(i + 1, len(mat)):
                this_row_first = mat[rr][i]
                scalarMultiple = -1 * this_row_first / first_row_first_col
                k[rr, i] =  scalarMultiple
                for cc in range(i, len(mat[0])):
                    mat[rr][cc] += mat[i][cc] * scalarMultiple
            break

    # At the end of the forward step.
    B = np.array(mat)

    # Now reduce.
    for i in range(min(len(mat), len(mat[0])) - 1, -1, -1):
        # Divide the last non-zero row by the first non-zero entry.
        first_elem_col = -1
        first_elem = -1
        for c in range(len(mat[0])):
            if mat[i][c] == 0:
                continue
            if first_elem_col == -1:
                first_elem_col = c
                first_elem = mat[i][c]
            mat[i][c] /= first_elem
        # Add multiples of this row so that all numbers above the leading 1 are zero.
        for r in range(i):
            this_row_above = mat[r][first_elem_col]
            scalarMultiple = -1 * this_row_above
            for cc in range(len(mat[0])):
                mat[r][cc] += mat[i][cc] * scalarMultiple
        # Disregard this row and continue.
    C = np.array(mat)
    return B, C

def get_k():
  global k
  return k

In [ ]:
def inv(A):
  if len(A) == 2:
    scale = 1/determinant(A)
    return np.round(matrix_scale([[A[1][1], -1*A[0][1]],[-1*A[1][0], A[0][0]]], scale), 3)
  rws = len(A)
  origdet = determinant(A)
  cofactor_T = []
  if origdet != 0:
    for i in range(rws):
      cofacTrow = []
      for j in range(rws):
        firstmat = np.delete(np.array(A), i, axis = 0)
        submat = np.delete(firstmat, j, axis = 1)
        Cij = ((-1)**(i+j))*determinant(submat)
        cofacTrow.append(Cij)
      cofactor_T.append(cofacTrow)
    adjA = transpose(cofactor_T)
    return np.round(matrix_scale(adjA, 1/origdet),3)
  else:
    return None

In [ ]:
#
# MATRIX OPERATIONS
#

def matAdd(A,B):
  A_T = transpose(A)
  B_T = transpose(B)
  AplusB = []
  for i in range(len(A_T)):
    AplusB.append(vec_add(A_T[i], B_T[i]))
  return np.array(AplusB)

def matSub(A,B):
  negB = matrix_scale(B, -1)
  return matAdd(A,negB)

def matmult(A,B):
  B_T = transpose(B)
  return np.array([[dotProd(row, Brow) for Brow in B_T] for row in A])

def determinant(A):
    A_T = np.transpose(A)

    if isUpper(A) or isUpper(A_T):
      return np.round(math.prod([A[i][i] for i in range(len(A))]),0)

    elif len(A) == 2:
        return np.round(A[0][0] * A[1][1] - A[0][1] * A[1][0], 0)
    else:
      L, U = rref(copy.deepcopy(A))
      detL = np.round(math.prod([L[i][i] for i in range(len(L))]),0)
      detU = np.round(math.prod([U[i][i] for i in range(len(U))]),0)
      return detL*detU


In [ ]:
QR([[1,2,3],[1,2,3],[1,2,3]])

In [ ]:
#
# TESTING
#


def generate_random_matrix(rows, cols):
    return [[random.randint(1, 10) for _ in range(cols)] for _ in range(rows)]
def generate_square_matrix(rows):
  cols = rows
  return [[random.randint(1, 10) for _ in range(cols)] for _ in range(rows)]

def test_matmult(num_tests):
    passed_cases = 0
    for i in range(num_tests):
        rows_A = random.randint(10, 14)
        cols_A = random.randint(10, 14)
        rows_B = cols_A  # Number of columns in A must match the number of rows in B
        cols_B = random.randint(10, 14)

        A = generate_random_matrix(rows_A, cols_A)
        B = generate_random_matrix(rows_B, cols_B)

        expected_output = np.dot(A, B)

        result = matmult(A, B)

        if np.array_equal(result, expected_output):
          passed_cases += 1
    return 'Out of ' + str(num_tests) + ' cases, ' + str(np.round((100*passed_cases/num_tests), 0)) + '% passed.'

def test_rank(num_tests):
  passed_cases = 0
  for i in range(num_tests):
        rows_A = random.randint(3, 5)
        cols_A = random.randint(3, 5)
        rows_B = cols_A  # Number of columns in A must match the number of rows in B
        cols_B = random.randint(3, 5)

        A = generate_random_matrix(rows_A, cols_A)

        expected_output = np.linalg.matrix_rank(A)

        r = rank(A)

        if r == expected_output:
          passed_cases += 1
  return 'Out of ' + str(num_tests) + ' cases, ' + str(np.round((100*passed_cases/num_tests), 0)) + '% passed.'

def test_inv(num_tests):
    passed = 0
    singular = 0


    for test in range(num_tests):
        rows = random.randint(3, 3)
        A = generate_square_matrix(rows)


        if np.linalg.det(A) != 0:
            expected_result = np.linalg.inv(A)
            my_result = inv(A)


            if my_result is not None:  # Check if matrix is invertible
                expected_result = expected_result.astype(np.float64)
                my_result = my_result.astype(np.float64)


                if np.allclose(expected_result, my_result, atol = .9):
                    passed += 1


            else:
                singular += 1


    num_inv = num_tests - singular
    passPercent = round(100 * (passed / num_inv))
    print('Out of', num_tests, 'test cases,', singular, 'were singular.')
    print('Out of', num_inv, 'test cases,', str(passed), 'passed, which is about', str(passPercent) + '%.' )

In [ ]:
constant = False
def leastSquares(A,b):
  if len(A[0]) == 2:
    A_T = transpose(A)
    A_T_A = matmult(A_T, A)
    Ab = matmult(A_T, b)
    inverse = inv(A_T_A)
    x = matmult(inverse, Ab)
    return np.array(x)
  else:
    A_T = transpose(A)
    A_T_A = matmult(A_T, A)
    A_Tb = matmult(A_T, b)
    inv = np.linalg.inv(A_T_A)
    x = np.dot(inv, A_Tb)
    return x


def leastSquaresError(A,b):
  g = leastSquares(A,b)
  bminAx = np.array(b) - matmult(A, g)
  return round(magnitude(bminAx)[0],3)

def add_constant(A):
  global constant
  A_T = transpose(A)
  constRow = [1]*len(A)
  sortedA_T = sorted(A_T, reverse = True )
  sortedA_T.append(constRow)
  fin = sorted(sortedA_T)
  constant = True
  return transpose(fin)

def getboo():
  global constant
  return constant


In [ ]:
def best_fitOLS(A, b):
  hasConst = getboo()
  if hasConst:
    if len(A[0]) > 2:
        x = leastSquares(A, b)
        npA = np.array(A)
        npB = np.array(b)
        predictedData = matmult(A, x.tolist())
        fig, ax = plt.subplots()
        ax.scatter(npB, predictedData, color= 'red')
        plt.xlabel('Observed')
        plt.ylabel('Predicted')
    else:
        x = leastSquares(A, b)
        npA = np.array(A)
        npB = np.array(b)
        axel = matmult(A, x.tolist())
        fig, ax = plt.subplots()
        newA = np.delete(npA, 0, axis = 1)
        ax.scatter(newA, npB)
        ax.plot(newA, axel, color= 'red')
        plt.xlabel('x')
        plt.ylabel('y')
  else:
    if len(A[0]) != 1:
        x = leastSquares(A, b)
        npA = np.array(A)
        npB = np.array(b)
        predictedData = matmult(A, x.tolist())
        fig, ax = plt.subplots()
        ax.scatter(npB, predictedData)
        plt.xlabel('Observed')
        plt.ylabel('Predicted')

    else:
        x = leastSquares(A, b)
        npA = np.array(A)
        npB = np.array(b)
        axel = matmult(A, x.tolist())
        fig, ax = plt.subplots()
        ax.scatter(npA, npB)
        ax.plot(npB, axel)
        plt.xlabel('x')
        plt.ylabel('y')


In [ ]:
def is_diagonal(matrix, atol = 0):
    n = len(matrix)

    # Check if matrix and its transpose are both in upper triangular form
    is_upper_triangular = all(all(matrix[i][j] == 0 for j in range(i + 1, n)) for i in range(n))
    is_transpose_upper_triangular = all(all(matrix[j][i] == 0 for j in range(i + 1, n)) for i in range(n))

    if is_upper_triangular and is_transpose_upper_triangular:
        # Check if off-diagonal elements of R are close to zero
        for i in range(n):
            for j in range(i + 1, n):
                if abs(matrix[i][j]) > 1e-6:
                    return False
        return True

    # Check if non-diagonal elements are very close to zero
    for i in range(n):
        for j in range(n):
            if i != j and abs(matrix[i][j]) > atol:
                return False

    return True



In [ ]:
def eigenvalues(A):
  if len(A) == 2:
    a = A[0][0]
    b = A[0][1]
    c = A[1][0]
    d = A[1][1]
    g = (a+b)
    n = a*b - b*c
    det = (g**2) - (4*n)
    if det >= 0:
      lam1 = g + np.sqrt(det)
      lam2 = g - np.sqrt(det)
      return [lam1, lam2]
    else:
      lam1 = str(g) + ' + ' + np.sqrt(abs(det)) + 'i'
      lam2 = str(g) + ' - ' + np.sqrt(abs(det)) + 'i'
      return [lam1, lam2]


  if is_diagonal(A, 1e-1):
    return [np.round(A[i][i],0) for i in range(len(A))]
  else:
    Q, R = QR(A)
    Anew = matmult(R,Q)
    return eigenvalues(Anew)

In [ ]:
matmult

In [ ]:
eigenvalues([[1,1],[1,2]])

[4.0, 0.0]

In [ ]:
scores = pd.read_csv('SATvsGPA.csv')
varSAT = scores['SAT'].var()
varGPA = scores['GPA'].var()
satbygpa = scores['SAT']*scores['GPA']
covSG = satbygpa.mean() - scores['SAT'].mean()*scores['GPA'].mean()
print('R^2 =',(covSG**2)/(varSAT*varGPA))
SAT = scores['SAT'].tolist()
GPA = scores['GPA'].tolist()
newSAT = transpose([SAT])
newGPA = transpose([GPA])
newSAT = add_constant(newSAT)
best_fitOLS(newSAT, newGPA)

FileNotFoundError: ignored